In [1]:
import sys
sys.path.append('../')

from src.layers.linear import Linear
from src.activation.functional import ReLU, Softmax, MSELoss
from src.optim.optim import SGD
from src.utils.template import BaseClass

import torch
from torch import nn

import numpy as np

In [2]:
class MLP(BaseClass):
    def __init__(self, input_size, output_size, hidden_size):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        self.layers = [
            Linear(input_size, hidden_size),
            ReLU(), 
            Linear(hidden_size, output_size),
            Softmax()
        ]

    def forward(self, x: np.ndarray) -> np.ndarray:
        assert len(self.layers) > 0
        for layer in self.layers:
            x = layer(x)
        return x

In [3]:
model = MLP(input_size=5, output_size=10, hidden_size=10)

In [4]:
inputs = np.array([[0.5, 0.7, 0.3, 0.1, 0.8],
                   [0.1, 0.6, 0.9, 0.01, 0.4]])
gt = np.array([2, 6])

In [5]:
preds = model.forward(inputs)
preds

array([[0.0811569 , 0.05634413, 0.13046026, 0.08205761, 0.11503573,
        0.08182587, 0.12946397, 0.1299204 , 0.11143506, 0.08230008],
       [0.08583283, 0.0670484 , 0.11323085, 0.0784106 , 0.13116556,
        0.08165283, 0.12084914, 0.13817391, 0.09952953, 0.08410635]])

In [6]:
cls = np.argmax(preds, axis=1)
cls

array([2, 7])

In [7]:
my_loss = MSELoss()
loss = my_loss(cls, gt)
loss_grad = my_loss.backward()

In [8]:
loss_grad

array([0., 1.])

In [9]:
optimizer = SGD(learning_rate=0.05)
optimizer.step(model, my_loss)

ValueError: shapes (20,1) and (20,1) not aligned: 1 (dim 1) != 20 (dim 0)